# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter


# Read the CSV and Perform Basic Data Cleaning

In [3]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [4]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv.zip')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N
5,25000.0,0.1797,634.43,MORTGAGE,90000.0,Source Verified,Mar-2019,low_risk,n,18.49,...,93.1,66.7,0.0,0.0,550717.0,418934.0,18800.0,369051.0,N,N
6,20400.0,0.2000,540.48,RENT,51315.0,Source Verified,Mar-2019,low_risk,n,25.63,...,100.0,0.0,1.0,0.0,40023.0,23469.0,3500.0,27523.0,N,N
7,10000.0,0.1557,349.46,OWN,66000.0,Source Verified,Mar-2019,low_risk,n,9.00,...,100.0,16.7,0.0,0.0,29300.0,22377.0,8400.0,12500.0,N,N
8,14000.0,0.0881,443.96,RENT,45000.0,Not Verified,Mar-2019,low_risk,n,10.24,...,100.0,0.0,0.0,0.0,25100.0,9724.0,12100.0,3000.0,N,N
9,10000.0,0.1474,345.39,RENT,63295.0,Verified,Mar-2019,low_risk,n,39.34,...,97.2,71.4,0.0,0.0,119822.0,102924.0,15300.0,80722.0,N,N


In [5]:
#Ecode variables using `LabelEncoder` from sklearn.preprocessing, after which we instantiate the `LabelEncoder()` object, then run a `.fit()` followed by `.transform()`. The results are stored in a new variable `encoded_y` and inserted into the DataFrame.

columns_to_encode = ['pymnt_plan', 'hardship_flag','initial_list_status', 'issue_d', 'home_ownership','verification_status', 'application_type', 'debt_settlement_flag', 'recoveries', 'collection_recovery_fee', 'acc_now_delinq', 'num_tl_120dpd_2m', 'num_tl_30dpd', 'policy_code','tax_liens', 'next_pymnt_d']  

for column in columns_to_encode:
    encoded_data = pd.get_dummies(df[column], columns= [column], prefix='Encoded')
    
    df = df.merge(encoded_data, how = "inner", left_index=True, right_index = True)

df = df.drop(columns = columns_to_encode)
df

,loan_amnt,int_rate,installment,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,Encoded_N_y,Encoded_0.0_x,Encoded_0.0_y,Encoded_0.0_x,Encoded_0.0_y,Encoded_0.0_x,Encoded_1.0,Encoded_0.0_y,Encoded_Apr-2019,Encoded_May-2019
0,10500.0,0.1719,375.35,66000.0,low_risk,27.24,0.0,0.0,8.0,0.0,...,1,1,1,1,1,1,1,1,0,1
1,25000.0,0.2000,929.09,105000.0,low_risk,20.23,0.0,0.0,17.0,1.0,...,1,1,1,1,1,1,1,1,0,1
2,20000.0,0.2000,529.88,56000.0,low_risk,24.26,0.0,0.0,8.0,0.0,...,1,1,1,1,1,1,1,1,0,1
3,10000.0,0.1640,353.55,92000.0,low_risk,31.44,0.0,1.0,10.0,1.0,...,1,1,1,1,1,1,1,1,0,1
4,22000.0,0.1474,520.39,52000.0,low_risk,18.76,0.0,1.0,14.0,0.0,...,1,1,1,1,1,1,1,1,0,1
5,25000.0,0.1797,634.43,90000.0,low_risk,18.49,1.0,1.0,14.0,0.0,...,1,1,1,1,1,1,1,1,0,1
6,20400.0,0.2000,540.48,51315.0,low_risk,25.63,0.0,2.0,8.0,1.0,...,1,1,1,1,1,1,1,1,0,1
7,10000.0,0.1557,349.46,66000.0,low_risk,9.00,0.0,0.0,11.0,0.0,...,1,1,1,1,1,1,1,1,0,1
8,14000.0,0.0881,443.96,45000.0,low_risk,10.24,0.0,1.0,9.0,0.0,...,1,1,1,1,1,1,1,1,0,1
9,10000.0,0.1474,345.39,63295.0,low_risk,39.34,0.0,0.0,27.0,0.0,...,1,1,1,1,1,1,1,1,0,1


# Split the Data into Training and Testing

In [6]:
# Create our features
X = df.drop(columns='loan_status')

# Create our target
y = df['loan_status'].to_frame()

In [7]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,Encoded_N_y,Encoded_0.0_x,Encoded_0.0_y,Encoded_0.0_x,Encoded_0.0_y,Encoded_0.0_x,Encoded_1.0,Encoded_0.0_y,Encoded_Apr-2019,Encoded_May-2019
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.0,68817.0,68817.0,68817.0,68817.0,68817.0,68817.0,68817.0,68817.000000,68817.000000
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.383161,0.616839
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.486161,0.486161
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.000000,0.000000
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.000000,0.000000
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.000000,1.000000
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.000000
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.000000


In [8]:
# Check the balance of our target values
y['loan_status'].value_counts()


low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [9]:
y['loan_status'].values


array(['low_risk', 'low_risk', 'low_risk', ..., 'low_risk', 'low_risk',
       'low_risk'], dtype=object)

In [10]:
# Create X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)


In [11]:
#y_test['loan_status'].value_counts()

In [12]:
X_train = X_train.values
X_test = X_test.values

# Scaling

In [13]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [14]:
# Resample the training data with the RandomOversampler

from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train_scaled, y_train)
print(Counter(y_resampled))

Counter({'loan_status': 1})


In [15]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(random_state=1, solver='lbfgs')
model.fit(X_resampled, y_resampled)

y_predicted = model.predict(X_test_scaled)

In [16]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

balanced_accuracy_score(y_test, y_predicted)

0.8375558030545804

In [17]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_predicted)

array([[   84,    17],
       [ 2678, 14426]])

In [18]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_predicted))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.83      0.84      0.06      0.84      0.70       101
   low_risk       1.00      0.84      0.83      0.91      0.84      0.70     17104

avg / total       0.99      0.84      0.83      0.91      0.84      0.70     17205



### SMOTE Oversampling

In [38]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=1)
X_resampled_smote, y_resampled_smote = smote.fit_resample(X_train_scaled, y_train)
print(Counter(y_resampled_smote))

Counter({'loan_status': 1})


In [39]:
# Train the Logistic Regression model using the resampled data
model.fit(X_resampled_smote, y_resampled_smote)

y_predicted_smote = model.predict(X_test_scaled)

In [40]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_predicted_smote)

0.8388510243681058

In [41]:
# Display the confusion matrix

confusion_matrix(y_test, y_predicted_smote)

array([[   82,    19],
       [ 2295, 14809]])

In [42]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_predicted_smote))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.81      0.87      0.07      0.84      0.70       101
   low_risk       1.00      0.87      0.81      0.93      0.84      0.71     17104

avg / total       0.99      0.87      0.81      0.92      0.84      0.71     17205



# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [44]:
# Resample the data using the ClusterCentroids resampler
from imblearn.under_sampling import ClusterCentroids

cluster_centroids = ClusterCentroids(random_state=1)
X_resampled_cc, y_resampled_cc = cluster_centroids.fit_resample(X_train_scaled, y_train)
print(Counter(y_resampled_cc))

Counter({'loan_status': 1})


In [45]:
# Train the Logistic Regression model using the resampled data
model.fit(X_resampled_cc, y_resampled_cc)

y_predicted_cc = model.predict(X_test_scaled)

In [46]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_predicted_cc)

0.8215575767118339

In [47]:
# Display the confusion matrix
confusion_matrix(y_test, y_predicted_cc)

array([[   89,    12],
       [ 4072, 13032]])

In [48]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_predicted_cc))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.02      0.88      0.76      0.04      0.82      0.68       101
   low_risk       1.00      0.76      0.88      0.86      0.82      0.66     17104

avg / total       0.99      0.76      0.88      0.86      0.82      0.66     17205



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [ ]:
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN

rebalanced = SMOTEENN(random_state=1)

X_rebalanced, y_rebalanced = rebalanced.fit_resample(X_train_scaled, y_train)
print(Counter(y_rebalanced.loan_status))

In [ ]:
# Train the Logistic Regression model using the resampled data
model.fit(X_rebalanced, y_rebalanced)
y_predicted_smoteen = model.predict(X_test_scaled)

In [ ]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_predicted_smoteen)

In [ ]:
# Display the confusion matrix
confusion_matrix(y_test, y_predicted_smoteen)

In [ ]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_predicted_smoteen))

# Linear Support Vector Classification

In [ ]:
from sklearn.svm import LinearSVC

svc = LinearSVC(random_state=1)

svc.fit(X_rebalanced, y_rebalanced)
predicted_svc = svc.predict(X_test_scaled)

In [ ]:
print(confusion_matrix(y_test, predicted_svc))


In [ ]:
print('Balanced Accuracy Score for SVC is :')
balanced_accuracy_score(y_test, predicted_svc)

In [ ]:
print(classification_report_imbalanced(y_test, predicted_svc))